In [1]:
!pip install pandas-datareader

*Library**

In [2]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [3]:
tf.__version__

'2.2.0-rc3'

In [0]:
class AI_Trader():
  def __init__(self, state_size, action_space=3,model_name="Ai_trader"):
    self.state_size=state_size
    self.action_space=action_space
    self.memory=deque(maxlen=2000)
    self.inventory=[]
    self.model_name = model_name
    self.gamma = 0.95
    self.epsilon=1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
    self.model = self.model_builder()

  def model_builder(self):
    model=tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=32,activation='relu',input_dim=self.state_size))
    model.add(tf.keras.layers.Dense(units=64,activation='relu'))
    model.add(tf.keras.layers.Dense(units=128,activation='relu'))
    model.add(tf.keras.layers.Dense(units=self.action_space,activation='linear'))
    model.compile(loss='mse',optimizer = tf.keras.optimizers.Adam(lr=0.001))
    return model

  def trade(self,state):
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    actions = self.model.predict(state)
    return np.argmax(actions[0])

  def batch_train(self, batch_size):
    batch=[]
    for i in range(len(self.memory)- batch_size + 1,len(self.memory)):
      batch.append(self.memory[i])

    for state,action,reward,next_state, done in batch:
      reward = reward
      if not done:
        reward= reward + self.gamma*np.amax(self.model.predict(next_state)[0])
      target = self.model.predict(state)
      target[0][action]=reward

      self.model.fit(state,target,epochs=1, verbose=0)
    if self.epsilon> self.epsilon_final:
      self.epsilon = self.epsilon_decay


In [0]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [0]:
def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

In [0]:
def dataset_loader(stock_name):
  
  #Complete the dataset loader function
  dataset = data_reader.DataReader(stock_name, data_source="yahoo")
  
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  
  close = dataset['Close']

  return close

In [10]:
dataset = data_reader.DataReader('AAPL', data_source="yahoo")
dataset

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2015-04-24,130.630005,129.229996,130.490005,130.279999,44525900.0,119.891411
2015-04-27,133.130005,131.149994,132.309998,132.649994,96954200.0,122.072395
2015-04-28,134.539993,129.570007,134.460007,130.559998,118924000.0,120.149078
2015-04-29,131.589996,128.300003,130.160004,128.639999,63386100.0,118.382187
2015-04-30,128.639999,124.580002,128.639999,125.150002,83195400.0,115.170479
...,...,...,...,...,...,...
2020-04-16,288.200012,282.350006,287.380005,286.690002,39281300.0,286.690002
2020-04-17,286.950012,276.859985,284.690002,282.799988,53812500.0,282.799988
2020-04-20,281.679993,276.850006,277.950012,276.929993,32503800.0,276.929993


In [0]:
def state_creator(data, timestep, window_size):
  
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
    
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
  return np.array([state])

In [0]:
 stock_name = "AAPL"
data = dataset_loader(stock_name)

In [0]:
window_size = 10
episodes = 1000

batch_size = 32
data_samples = len(data) - 1

In [14]:
data


Date
2015-04-24    130.279999
2015-04-27    132.649994
2015-04-28    130.559998
2015-04-29    128.639999
2015-04-30    125.150002
                 ...    
2020-04-16    286.690002
2020-04-17    282.799988
2020-04-20    276.929993
2020-04-21    268.369995
2020-04-22    273.380005
Name: Close, Length: 1258, dtype: float64

In [0]:
trader = AI_Trader(window_size)

In [16]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


In [0]:
for episode in range(1, episodes + 1):
  
  print("Episode: {}/{}".format(episode, episodes))
  
  state = state_creator(data, 0, window_size + 1)
  
  total_profit = 0
  trader.inventory = []
  
  for t in tqdm(range(data_samples)):
    
    action = trader.trade(state)
    
    next_state = state_creator(data, t+1, window_size + 1)
    reward = 0
    
    if action == 1: #Buying
      trader.inventory.append(data[t])
      print("AI Trader bought: ", stocks_price_format(data[t]))
      
    elif action == 2 and len(trader.inventory) > 0: #Selling
      buy_price = trader.inventory.pop(0)
      
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price) )
      
    if t == data_samples - 1:
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("########################")
    
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
      
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))
    

  0%|          | 0/1257 [00:00<?, ?it/s]

Episode: 1/1000
AI Trader bought:  $ 130.279999
AI Trader sold:  $ 132.649994  Profit: $ 2.369995
AI Trader bought:  $ 130.559998
AI Trader bought:  $ 128.639999
AI Trader bought:  $ 125.150002
AI Trader sold:  $ 128.699997  Profit: - $ 1.860001
AI Trader bought:  $ 125.800003
AI Trader sold:  $ 125.010002  Profit: - $ 3.629997
AI Trader sold:  $ 127.620003  Profit: $ 2.470001
AI Trader bought:  $ 126.320000
AI Trader bought:  $ 125.870003
AI Trader sold:  $ 126.010002  Profit: $ 0.209999
AI Trader bought:  $ 128.949997
AI Trader bought:  $ 128.770004
AI Trader sold:  $ 130.190002  Profit: $ 3.870003
AI Trader sold:  $ 132.039993  Profit: $ 6.169991
AI Trader sold:  $ 130.279999  Profit: $ 1.330002
AI Trader sold:  $ 129.960007  Profit: $ 1.190002
AI Trader bought:  $ 130.119995
AI Trader sold:  $ 129.360001  Profit: - $ 0.759995


  3%|▎         | 33/1257 [00:05<03:09,  6.47it/s]

AI Trader bought:  $ 128.589996


  3%|▎         | 34/1257 [00:07<19:33,  1.04it/s]

...

 30%|██▉       | 372/1257 [16:15<42:01,  2.85s/it]

AI Trader sold:  $ 116.980003  Profit: $ 3.930000


 30%|██▉       | 373/1257 [16:18<42:36,  2.89s/it]

AI Trader sold:  $ 117.629997  Profit: $ 3.570000


 30%|██▉       | 374/1257 [16:21<42:17,  2.87s/it]